In [1]:
import pandas as pd
import numpy as np

import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [3]:
data = pd.read_csv('cyber_bully_cleaned.csv')
data.head()

,Unnamed: 0,tweet_text,cyberbullying_type
0,26156,"['andyeaston85', 'love', 'teaching', 'bullshit...",other_cyberbullying
1,15596,"['gyuk', 'youtuber', 'double', 'vile', 'jess',...",gender
2,19412,"['islam', 'declared', 'war', 'mankind', '1400'...",religion
3,44068,"['broconfessional', 'asheton3195', 'weak', 'fu...",ethnicity
4,7305,"['kat', 'andre', 'behaving', 'spiteful', 'vind...",not_cyberbullying


In [6]:
data = data[data["cyberbullying_type"] != "other_cyberbullying"]
data = data[["tweet_text", "cyberbullying_type"]]
data.head()

,tweet_text,cyberbullying_type
1,"['gyuk', 'youtuber', 'double', 'vile', 'jess',...",gender
2,"['islam', 'declared', 'war', 'mankind', '1400'...",religion
3,"['broconfessional', 'asheton3195', 'weak', 'fu...",ethnicity
4,"['kat', 'andre', 'behaving', 'spiteful', 'vind...",not_cyberbullying
5,"['hear', 'snowing', 'north', 'glad', 'made', '...",not_cyberbullying


### Word2Vec

In [7]:
import gensim.downloader as api
word2vec_model = api.load('word2vec-google-news-300')

In [18]:
#find unique words in the dataset
unique_words = set()
for tweet in data["tweet_text"]:
    print(tweet)
    for word in tweet:
        print(word)
    break

['gyuk', 'youtuber', 'double', 'vile', 'jess', 'phillips', 'rape', 'joke', 'leaping', 'defence', 'video', 'uploaded', 'thursday', 'april', '23', 'former', 'ukip', 'candidate', 'benjamin', 'jumped', 'defence', 'retired', 'gay', 'porn', 'actor', 'turned', 'men', 'http']
[
'
g
y
u
k
'
,
 
'
y
o
u
t
u
b
e
r
'
,
 
'
d
o
u
b
l
e
'
,
 
'
v
i
l
e
'
,
 
'
j
e
s
s
'
,
 
'
p
h
i
l
l
i
p
s
'
,
 
'
r
a
p
e
'
,
 
'
j
o
k
e
'
,
 
'
l
e
a
p
i
n
g
'
,
 
'
d
e
f
e
n
c
e
'
,
 
'
v
i
d
e
o
'
,
 
'
u
p
l
o
a
d
e
d
'
,
 
'
t
h
u
r
s
d
a
y
'
,
 
'
a
p
r
i
l
'
,
 
'
2
3
'
,
 
'
f
o
r
m
e
r
'
,
 
'
u
k
i
p
'
,
 
'
c
a
n
d
i
d
a
t
e
'
,
 
'
b
e
n
j
a
m
i
n
'
,
 
'
j
u
m
p
e
d
'
,
 
'
d
e
f
e
n
c
e
'
,
 
'
r
e
t
i
r
e
d
'
,
 
'
g
a
y
'
,
 
'
p
o
r
n
'
,
 
'
a
c
t
o
r
'
,
 
'
t
u
r
n
e
d
'
,
 
'
m
e
n
'
,
 
'
h
t
t
p
'
]


In [ ]:
embedding_matrix =[[0] * 300] #this matrix will contain the word embeddings for the top 5000 words #the first row will be the embedding for the padding token which is 0
word_indices = {} #this dictionary will map each word to its index in the embedding matrix, this is needed when we train the neural network, the input to the neural network will be the index of the word in the embedding matrix
idx = 1
for i, word in enumerate(top_5000_words):
    if word in word2vec_model.key_to_index:
        embedding_matrix.append(word2vec_model[word])
        word_indices[word] = idx
        idx += 1
    
    else:
        continue
embedding_matrix = np.array(embedding_matrix)
print(embedding_matrix.shape)